# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

'pytor'

# Result Filename

In [2]:
result_filename='Results_FaceApp.csv'

# Imports

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
import sys
import time
from PIL import Image
import csv
from time import sleep
from glob import glob
from pprint import pprint

## API Setting 

In [4]:
subscription_key = "761dd2a06888467797cd62e4024b816e"
endpoint="https://adf-vision.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
subscription_key = "875b75f011664840a6bd422532cb4e53"
endpoint="https://adf-face.cognitiveservices.azure.com/"
face_client=FaceClient(endpoint, CognitiveServicesCredentials(subscription_key))

## FaceApp

### Describe an Image

In [5]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.'Scarlett Johansson_fake.png'
'''
def describe_image(filename):
#     print("===== Describe an image - local =====")
    # Call API
    image= open(filename,"rb")
    description_results = computervision_client.describe_image_in_stream(image)

    # Get the captions (descriptions) from the response, with confidence level
#     print("Description of local image: ")
    if (len(description_results.captions) == 0):
#         print("No description detected.")
        return "No description detected."
    else:
        for caption in description_results.captions:
#             print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            return "'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100)

### Detect Domain-specific Content

In [6]:
'''
Detect Domain-specific Content - local
This example detects celebrites and landmarks in local images.
'''
def domain_specific_content(filename):
#     print("===== Detect Domain-specific Content - local =====")
    # Image of one or more celebrities
    image= open(filename,"rb")
    # image = image.read('Scarlett.png')

    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities", image)
    # Print detection results with name
#     print("Celebrities in the local image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
        return "",""
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
#             print(celeb["name"])
#             print(celeb["confidence"])
            return celeb["name"],celeb["confidence"]

In [14]:
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0]

In [8]:
def check_similarity(src_image, tar_image):
    # Detect a face in an image that contains a single face
    src_image_name = os.path.basename(src_image)
    detected_faces = face_client.face.detect_with_stream(open(src_image,"rb"))
    if not detected_faces:
        print('No face detected from image {}'.format(single_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(single_image_name))

    # Display the detected face ID in the first single-face image.
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
#     print('Detected face ID from', single_image_name, ':')
#     for face in detected_faces: 
#         print (face.face_id)
#     print()

    # Save this ID for use in Find Similar
    first_image_face_ID = detected_faces[0].face_id
    
    multi_image_name = os.path.basename(tar_image)
    detected_faces2 = face_client.face.detect_with_stream(open(tar_image,"rb"))
    if not detected_faces2:
        print('No face detected from image {}'.format(multi_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(multi_image_name))
    
    # Search through faces detected in group image for the single face from first image.
    # First, create a list of the face IDs found in the second image.
    second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
    
    if not similar_faces:
        print('No similar faces found in', multi_image_name, '.')
        return 0
    else:
        # Print the details of the similar faces detected
#         print('Similar faces found in', multi_image_name + ':')
        for face in similar_faces:
        #     print(face)
            return face.confidence
#             print('  Confidence:', face.confidence)
#             first_image_face_ID = face.face_id
#             # The similar face IDs of the single face image and the group image do not need to match, 
#             # they are only used for identification purposes in each image.
#             # The similar faces are matched using the Cognitive Services algorithm in find_similar().
#             face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
#             if face_info:
#                 print('  Face ID: ', first_image_face_ID)
#                 print('  Face rectangle:')
#                 print('    Left: ', str(face_info.face_rectangle.left))
#                 print('    Top: ', str(face_info.face_rectangle.top))
#                 print('    Width: ', str(face_info.face_rectangle.width))
#                 print('    Height: ', str(face_info.face_rectangle.height))

In [19]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity","Src Des", "Tar Des"])
file.close()
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/*")
count=0
skip=0
for actor in actors_list:
    count+=1

#     if skip ==0:
#         if os.path.basename(actor) =="June Lapine Aka Shoe0nHead":
#             skip=1
#         else:
#             continue
#     print(os.path.basename(actor))
#     break
    
    real_image=os.path.join("/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/real",
                                os.path.basename(actor)+".jpg")
    fake_images_list=glob(os.path.join(actor,"*"))
    print(real_image)
#     pprint(fake_images_list)
#     continue
    result_real_pred,result_real_conf=domain_specific_content(real_image)
    real_des=describe_image(real_image)
    src=get_name(real_image)
#     print("SRC:",src)
#     break
    for fake_image in fake_images_list:
        print(fake_image)
        tar=get_name(fake_image)
#         print("Tar:",tar)
#         break
        result_fake_pred,result_fake_conf=domain_specific_content(fake_image)
        fake_des=describe_image(fake_image)
        
        similarity=check_similarity(real_image,fake_image)
#             print(fake_image,src,tar,
#                   result_real_pred,result_real_conf,
#                   result_fake_pred,result_fake_conf,
#                   similarity,real_des,fake_des)
        file=open(result_filename, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow([fake_image,src,tar,
              result_real_pred,result_real_conf,
              result_fake_pred,result_fake_conf,
              similarity,real_des,fake_des])
        file.close()
#             sleep(10)
#             break
#         break
#     break
# file.close()
print("Total:",count)

/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/real/Emma Watson.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Emma Watson/IU.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Emma Watson/Emma Stone.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Emma Watson/Scarlett Johansson.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Emma Watson/Madhuri Dixit.jpg
No celebrities detected.
No similar faces found in Madhuri Dixit.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Emma Watson/Ariana Grande.jpg
No celebrities detected.
No similar faces found in Ariana Grande.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Emma Watson/Rihanna.jpg
No celebrities detected.
No similar faces found in Rihanna.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/E

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Barack Obama.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Chris Hemsworth.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Cristiano Ronaldo.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Chris Evans.jpg
No celebrities detected.
No similar faces found in Chris Evans.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Donald Trump.jpg
No celebrities detected.
No similar faces found in Donald Trump.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Shah Rukh Khan.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Brad Pitt/Daniel Radcliffe.jpg
No celebrities detected.
No similar faces found in Daniel Radclif

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Tom Cruise.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Brad Pitt.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Barack Obama.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Chris Hemsworth.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Cristiano Ronaldo.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Chris Evans.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Donald Trump.jpg
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Daniel Radcliffe/Shah Ru

No celebrities detected.
No similar faces found in Ariana Grande.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Angelina Jolie/Rihanna.jpg
No celebrities detected.
No similar faces found in Rihanna.jpg .
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Angelina Jolie/Tamannaah.jpg
/home/sha/project/adversarial_deepfake/src/Similarity/FaceApp/fake/Angelina Jolie/Billie Eilish.jpg
Total: 20
